In [4]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils import data
def prepare_data(batch_size):
    torch.set_default_tensor_type(torch.cuda.FloatTensor)

    root_dir = "./data/train";
    train_set = datasets.ImageFolder(root = root_dir,
                transform = transforms.ToTensor())

    trainloader = data.DataLoader(train_set, batch_size = batch_size, shuffle = True)

    root_dir = "./data/test";

    train_set = datasets.ImageFolder(root = root_dir,
                transform = transforms.ToTensor())

    testloader = data.DataLoader(train_set, batch_size = batch_size, shuffle = True)
    classes = ('Natalie Portman', 'Julia Roberts', 'Neil Patrick Harris'
                    , 'Keifer Sutherland', 'Ben Stiller', 'Anne Hathaway'
                    , 'David Boreanaz', 'Jamie Foxx', 'Sofia Vergara'
                    , 'Elizabeth Banks', 'Jensen Ackles', 'Amy Adams'
                    , 'Kristen Bell', 'Bradley Cooper', 'Emily Deschanel'
                    , 'Zooey Deschanel', 'Jon Hamm', 'Scarlett Johansson'
                    , 'Blake Lively', 'Eva Longoria', 'Amy Poehler'
                    , 'Kristen Stewart', 'Kerry Washington', 'Leighton Meester'
                    , 'Olivia Wilde', 'Zac Efron', 'Miley Cyrus','Jim Parsons')
    return trainloader, testloader, classes
trainloader, testloader, classes = prepare_data(1)
#
print("train size: " + str(len(trainloader)))
#
print("test size: " + str(len(testloader)))

FileNotFoundError: [Errno 2] No such file or directory: './data/train'

In [ ]:
trainloader, testloader, classes = prepare_data(1)

import matplotlib.pyplot as plt
import numpy as np


# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(1)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def prepare_and_init_Net(number_layers_1, number_layers_2, number_layers_3):
    class Net(nn.Module):
        def __init__(self, device):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 6, 11)
            self.conv2 = nn.Conv2d(6, 12, 9)
            self.conv3 = nn.Conv2d(12, 24, 7)
            self.conv4 = nn.Conv2d(24, 30, 6)
            self.pool2 = nn.MaxPool2d(2, 2)
            self.conv2_bn_30 = nn.BatchNorm2d(30)
            self.conv2_bn_24 = nn.BatchNorm2d(24)
            self.conv2_bn_12 = nn.BatchNorm2d(12)
            self.conv2_bn_6 = nn.BatchNorm2d(6)
            # self.pool3 = nn.MaxPool2d(3,3)
            self.fc1 = nn.Linear(30 * 10 * 10, 240)       #coś tu się dzieje, nie jestem pewnien co
            self.fc2 = nn.Linear(240, 120)
            self.fc3 = nn.Linear(120, 28)

        def forward(self, x):
            # print(x.shape)
            x = self.pool2(F.relu(self.conv2_bn_6(self.conv1(x))))
            # print(x.shape)
            x = self.pool2(F.relu(self.conv2_bn_12(self.conv2(x))))

            x = self.pool2(F.relu(self.conv2_bn_24(self.conv3(x))))
            x = self.pool2(F.relu(self.conv2_bn_30(self.conv4(x))))
            # print(x.shape)
            # print("to tu?")
            x = x.view(-1, 30 * 10 * 10)
            # print(x.shape)
            x = F.relu(self.fc1(x))
            # print(x.shape)
            x = F.relu(self.fc2(x))
            # print(x.shape)
            x = self.fc3(x)
            # print(x.shape)
            return x


    net = Net(device='cuda:0')
    return net
# print(net.parameters())

In [ ]:
import torch.optim as optim

net = prepare_and_init_Net(6, 12, 24)

def crit_and_opt(learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    return criterion, optimizer

In [ ]:
def cuda_init():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    return device
# Assuming that we are on a CUDA machine, this should print a CUDA device:


In [ ]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
def accuracy(device, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Accuracy of the network on the 2772 test images: %f %%' % (
        100 * correct / total))
        return 100 * correct / total


In [ ]:
def learn(device, trainloader, testloader, optimizer, criterion):
    running_loss_for_optimizer = 0.0
    running_loss = 0.0
    loss_for_optimizer = 1.0
    learning_rate = 0.001
    counter=0
    for epoch in range(15):  # loop over the dataset multiple times


        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            # inputs, labels = data
            inputs, labels = data[0].to(device), data[1].to(device)
            # print(inputs.shape)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            # print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            running_loss_for_optimizer += loss.item()
            # if i % 4000 == 3999:    # print every 2000 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #           (epoch + 1, i + 1, running_loss / 4000))
            #     running_loss = 0.0
        print("running loss at end of epoch: ", running_loss_for_optimizer/len(trainloader))
        # if(accuracy(device, testloader) > 75):
        #     break
        accuracy(device, testloader)
        if(running_loss_for_optimizer/len(trainloader) < loss_for_optimizer):
            learning_rate /= 2
            loss_for_optimizer /= 2
            criterion, optimizer = crit_and_opt(learning_rate)
            print( "devaluating optimizer, current value: ", loss_for_optimizer)
        running_loss_for_optimizer = 0.0
        running_loss = 0.0


    print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
batch_size = 1
while(batch_size < 513):
    print("New batch size testing, batch size: ", batch_size)
    trainloader, testloader, classes = prepare_data(batch_size)
    device = cuda_init()
    net = prepare_and_init_Net(6, 12, 24)
    criterion, optimizer = crit_and_opt(0.001)
    learn(device, trainloader, testloader, optimizer, criterion)
    batch_size *= 2
